In [1]:
import time
import numpy as np
import threading

from scipy.signal import find_peaks
import sounddevice as sd

In [2]:
# Initial silent duration in seconds
silent_duration = 0.1 

#Last time a backchannel was played, initialized to current time
last_backchannel_time = time.time()

# Minimum time between backchannels in seconds
BACKCHANNEL_GAP = 5.0  

# Sample rate in Hz - number of samples of audio carried per second.
SAMPLERATE = 44100 

# Mono audio
CHANNELS = 1

# Samples per block - number of audio samples processed at once.
BLOCKSIZE = 1024  


# Duration of silence before backchannel in seconds
# is the duration of silence that must be exceeded 
#before considering playing a backchannel sound again.
SILENT_THRESHOLD = 0.5  



# Autocorrelation function
"""
This function performs autocorrelation on an audio signal to help in 
pitch detection. Autocorrelation is a mathematical tool for finding 
repeating patterns, such as the presence of a periodic signal obscured by noise.
"""
def autocorrelate(signal):
    """function takes a signal, 
    which is a numpy array representing the audio waveform.
    """
    n = len(signal) # compute length of the signal
    mean = np.mean(signal) # compute mean of the signal
    
    # compute correlation of the signal with itself
    #which is essentially the sum of products of the signal values 
    #at different time shifts.
    correlation = np.correlate(signal - mean, signal - mean, mode='full')
    return correlation[n - 1:]



# Feature extraction function
def extract_features(audio_data):
    features = {}
    
    # Apply autocorrelation
    correlation_result = autocorrelate(audio_data)
    
    # Find peaks in the autocorrelation result
    peaks, _ = find_peaks(correlation_result, height=0.1)
    
    if len(peaks) == 0:
        features['F0'] = 0
    else:
        # Take the first peak as the fundamental period
        period = peaks[0]
        
        # Calculate the fundamental frequency (F0)
        features['F0'] = 44100 / period

    print(int(features['F0']), end=" ")
    return features


# Soft beep function (as before)
def play_soft_beep():
    # Parameters for the beep
    frequency = 440  # Frequency of the beep (in Hz)
    duration = 0.2   # Duration of the beep (in seconds)
    amplitude = 0.1  # Amplitude of the beep (0.0 to 1.0 for soft to loud)

    # Generate the time points
    sample_rate = 44100  # Samples per second
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)

    # Generate the beep as a sine wave
    beep = amplitude * np.sin(2 * np.pi * frequency * t)

    print(" BC ", end="\n\n")
    # Play the beep
    sd.play(beep, sample_rate)
    sd.wait()  # Wait until the sound is done playing

    
# Function to handle backchannel with non-blocking behavior
def handle_backchannel():
    global last_backchannel_time
    current_time = time.time()
    if current_time - last_backchannel_time > BACKCHANNEL_GAP:
        # Play backchannel sound in a non-blocking manner
        threading.Thread(target=play_soft_beep).start()
        # Update the timestamp for the last backchannel
        last_backchannel_time = current_time

        
# Callback function for real-time audio processing
def callback(indata, frames, time, status):
    global silent_duration
    audio_data = np.frombuffer(indata, dtype=np.float32)
    features = extract_features(audio_data)
    
    # If fundamental frequency (F0) is zero, increment the silent duration
    if features['F0'] == 0:
        silent_duration += frames / SAMPLERATE
    else:
        silent_duration = 0  # Reset silent duration if F0 is not zero

    # Check if silent duration exceeds threshold and handle backchannel accordingly
    if silent_duration >= SILENT_THRESHOLD:
        handle_backchannel()
        silent_duration = 0  # Reset silent duration

        
# Start the audio stream
print("Streaming... Press Ctrl+C to stop.")
with sd.InputStream(callback=callback, 
                    channels=CHANNELS, 
                    samplerate=SAMPLERATE, 
                    blocksize=BLOCKSIZE):
    # Keep the stream alive
    while True:
        time.sleep(0.1)ç

Streaming... Press Ctrl+C to stop.
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 120 196 506 495 484 8820 7350 6300 6300 397 464 450 109 0 490 525 518 495 120 4009 4900 4410 0 445 490 495 490 122 0 0 0 454 479 119 0 0 0 501 501 512 490 120 121 0 0 0 0 0 537 518 512 118 120 0 0 0 404 416 459 474 464 441 484 501 525 580 580 525 495 459 116 122 122 0 0 0 130 243 116 512 580 595 639 689 711 648 639 572 572 108 109 107 0 0 0 518 518 551 140 0 0 0 0 0 0 474 436 490 138 0 0 0 450 469 512 474 111 109 114 121 464 132 135 142 147 1575 1422 157 147 408 479 572 604 621 630 621 177 143 141 207 558 588 648 658 658 689 711 139 420 390 544 265 130 130 257 321 119 113 558 639 630 595 588 658 113 114 0 0 0 0 113 454 459 525 621 116 6300 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0  BC 

0 0 0 0 0 0 441 441 441 464 454 441 441 441 454 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

KeyboardInterrupt: 